<h1>Multi-agent

multi-agent可用于处理一些single agent不好处理的问题<br>
两种常用的框架：supervisor/swarm<br>
supervisor：一个父agent管理几个子agent<br>
swarm：多个agent间相互作用互相影响

<h1>Supervisor</h1>

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

In [2]:
@tool
def book_hotel(hotel_name: str):
    """Book a hotel"""
    return f"Successfully booked a stay at {hotel_name}."
@tool
def book_flight(from_airport: str, to_airport: str):
    """Book a flight"""
    return f"Successfully booked a flight from {from_airport} to {to_airport}."


In [12]:
model = ChatOpenAI(
    model="gpt-3.5-turbo", 
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://free.v36.cm/v1"
)

In [13]:
#航班预订智能代理
flight_assistant = create_react_agent(
    model="gpt-3.5-turbo",
    tools=[book_flight],
    prompt="You are a flight booking assistant",
    name="flight_assistant"
)
#酒店预订智能代理
hotel_assistant = create_react_agent(
    model="gpt-3.5-turbo",
    tools=[book_hotel],
    prompt="You are a hotel booking assistant",
    name="hotel_assistant"
)

In [14]:
#创建主管系统，管理子系统
supervisor = create_supervisor(
    agents=[flight_assistant, hotel_assistant],
    model=ChatOpenAI(
    model="gpt-3.5-turbo", 
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://free.v36.cm/v1"
),
    prompt=(
        "You manage a hotel booking assistant and a"
        "flight booking assistant. Assign work to them."
    )
).compile()

In [17]:
for chunk in supervisor.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "book a flight from BOS to JFK and a stay at McKittrick Hotel"
            }
        ]
    }
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='book a flight from BOS to JFK and a stay at McKittrick Hotel', additional_kwargs={}, response_metadata={}, id='c933fcba-d190-4234-b439-150f510e4223'), AIMessage(content='I will assign the tasks to the respective assistants.\n\n**Flight Booking Assistant**: Please book a flight from Boston Logan International Airport (BOS) to John F. Kennedy International Airport (JFK).\n\n**Hotel Booking Assistant**: Please arrange a stay at the McKittrick Hotel.\n\nIf you have specific dates or preferences for the flight and hotel stay, please provide those details for a more accurate booking.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 34, 'total_tokens': 114, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_f9f5a388757c', 'id': 'chatcmpl-88cdf14d-15f7-4618-bcc6-722094b3f119', 'service_tier': None, 'finish

可以看出正常调用了llm但设定的tools又被忽略了，怎么会事呢

<h1>Swarm</h1>

In [26]:
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_swarm, create_handoff_tool

In [27]:
transfer_to_hotel_assistant = create_handoff_tool(
    agent_name="hotel_assistant",
    description="Transfer user to the hotel-booking assistant.",
)
transfer_to_flight_assistant = create_handoff_tool(
    agent_name="flight_assistant",
    description="Transfer user to the flight-booking assistant.",
)

In [31]:
flight_assistant = create_react_agent(
    model=model,
    tools=[book_flight, transfer_to_hotel_assistant],
    prompt="You are a flight booking assistant",
    name="flight_assistant"
)
hotel_assistant = create_react_agent(
    model=model,
    tools=[book_hotel, transfer_to_flight_assistant],
    prompt="You are a hotel booking assistant",
    name="hotel_assistant"
)


In [32]:
swarm = create_swarm(
    agents=[flight_assistant, hotel_assistant],
    default_active_agent="flight_assistant"
).compile()

In [33]:
for chunk in swarm.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "book a flight from BOS to JFK and a stay at McKittrick Hotel"
            }
        ]
    }
):
    print(chunk)
    print("\n")

{'flight_assistant': {'messages': [HumanMessage(content='book a flight from BOS to JFK and a stay at McKittrick Hotel', additional_kwargs={}, response_metadata={}, id='7f78358a-621a-4b48-9206-c916787fd919'), AIMessage(content="I’m unable to book flights or accommodations directly. However, I can guide you on how to do it.\n\n### Booking a Flight from Boston Logan International Airport (BOS) to John F. Kennedy International Airport (JFK):\n\n1. **Visit a Flight Booking Website**: Go to popular travel sites like Expedia, Kayak, Google Flights, or the airline's website directly (e.g., Delta, JetBlue).\n  \n2. **Enter Your Travel Details**:\n   - Departure: BOS\n   - Arrival: JFK\n   - Select your travel dates.\n   - Choose the number of passengers.\n\n3. **Search for Flights**: Click on the search button to view available flights.\n\n4. **Select Your Flight**: Review the options and select the flight that best fits your schedule and budget.\n\n5. **Provide Passenger Information**: Enter t

<h1>Handoffs</h1><br>
可将智能体控制权转交为另一个智能体

In [34]:
from typing import Annotated
from langchain_core.tools import tool, InjectedToolCallId
from langgraph.prebuilt import create_react_agent, InjectedState
from langgraph.graph import StateGraph, START, MessagesState
from langgraph.types import Command

In [ ]:
def create_handoff_tool(*, agent_name: str, description: str | None = None):
    # *强制使用关键词参数
    name = f"transfer_to_{agent_name}"  ##f表示格式化字符，等于name = "transfer_to_{}".format(agent_name)
    description = description or f"Transfer to {agent_name}"

    @tool(name, description=description)
    def handoff_tool(
        state: Annotated[MessagesState, InjectedState],   #Annotated用于添加额外的标注信息
        tool_call_id: Annotated[str, InjectedToolCallId],
    ) -> Command:
        tool_message = {
            "role": "tool",
            "content": f"Successfully transferred to {agent_name}",
            "name": name,
            "tool_call_id": tool_call_id,
        }
        return Command(  
            goto=agent_name,  
            update={"messages": state["messages"] + [tool_message]},  
            graph=Command.PARENT,  
        )
    return handoff_tool

In [36]:
transfer_to_hotel_assistant = create_handoff_tool(
    agent_name="hotel_assistant",
    description="Transfer user to the hotel-booking assistant.",
)
transfer_to_flight_assistant = create_handoff_tool(
    agent_name="flight_assistant",
    description="Transfer user to the flight-booking assistant.",
)

In [37]:
def book_hotel(hotel_name: str):
    """Book a hotel"""
    return f"Successfully booked a stay at {hotel_name}."

def book_flight(from_airport: str, to_airport: str):
    """Book a flight"""
    return f"Successfully booked a flight from {from_airport} to {to_airport}."

In [38]:
flight_assistant = create_react_agent(
    model=model,
    tools=[book_flight, transfer_to_hotel_assistant],
    prompt="You are a flight booking assistant",
    name="flight_assistant"
)
hotel_assistant = create_react_agent(
    model=model,
    tools=[book_hotel, transfer_to_flight_assistant],
    prompt="You are a hotel booking assistant",
    name="hotel_assistant"
)

In [42]:
# Define multi-agent graph
multi_agent_graph = (
    StateGraph(MessagesState)
    .add_node(flight_assistant)
    .add_node(hotel_assistant)
    .add_edge(START, "flight_assistant")
    .compile()
)

In [41]:
# Run the multi-agent graph
for chunk in multi_agent_graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "book a flight from BOS to JFK and a stay at McKittrick Hotel"
            }
        ]
    }
):
    print(chunk)
    print("\n")

{'flight_assistant': {'messages': [HumanMessage(content='book a flight from BOS to JFK and a stay at McKittrick Hotel', additional_kwargs={}, response_metadata={}, id='2a5314d2-8548-4483-be99-c7f8509b5da8'), AIMessage(content='I can’t book flights or accommodations directly, but I can guide you on how to do it. Here’s how you can book a flight from Boston Logan International Airport (BOS) to New York John F. Kennedy International Airport (JFK) and a stay at the McKittrick Hotel:\n\n### Step 1: Book Your Flight\n\n1. **Visit a Flight Booking Website**: Go to a travel website like Expedia, Kayak, Google Flights, or the airline\'s website.\n2. **Enter Your Details**:\n   - Departure: BOS (Boston Logan International Airport)\n   - Arrival: JFK (John F. Kennedy International Airport)\n   - Choose your travel dates and the number of passengers.\n3. **Search for Flights**: Click on the search button to find available flights.\n4. **Select a Flight**: Review the options and choose a flight tha

In [43]:
graph_png = multi_agent_graph.get_graph().draw_mermaid_png()
with open("multi_agent_graph.png", "wb") as f:
    f.write(graph_png)

工作流程：<br>
先激活fight_assistant->转移权限到hotel_assistant